In [26]:
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta

%matplotlib inline

In [27]:
# Set the figure parameters
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['figure.figsize'] = (15, 8)  # 图片大小
# plt.rcParams['figure.dpi'] = 300  # 显示图片的分辨率
plt.rcParams['savefig.dpi'] = 300  # 保存图片的分辨率

In [28]:
pd.set_option('display.max_columns', 500)

# 1 预处理

In [4]:
sales = pd.read_csv(
    "../data/daily-sales.csv", sep=',', parse_dates=['creation_date']).set_index(
    ['sales_cen_wid', 'customer_wid', 'item_code', 'creation_date'])

In [5]:
sales['qty'] = sales['item_qty'] - sales['return_qty']
sales.drop(['item_qty', 'return_qty'], axis=1, inplace=True)

In [6]:
# 每个销售中心每个客户每个产品每天的销量
sales = sales.unstack(level=-1).fillna(0)
sales.columns = sales.columns.get_level_values(1)

In [7]:
sales.head()

creation_date                              2018-01-01  2018-01-02  2018-01-03  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-04  2018-01-05  2018-01-06  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-07  2018-01-08  2018-01-09  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-10  2018-01-11  2018-01-12  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-13  2018-01-14  2018-01-15  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-16  2018-01-17  2018-01-18  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date                              2018-01-19  2018-01-20  2018-01-21  \
sales_cen_wid customer_wid item_code                                            
-1            599          31022010003988         0.0         0.0         0.0   
                           31022010003989         0.0         0.0         0.0   
                           31022010004009         0.0         0.0         0.0   
                           31022010004010         0.0         0.0         0.0   
                           31022010005109         0.0         0.0         0.0   

creation_date           

In [8]:
items = pd.read_csv("../processed_data/item-info.csv", sep=',').set_index('item_code')

In [9]:
items.head()

,sales_category_wid,productform,energyeffratiing,frequencytype
item_code,,,,
31026113Z12712,368,1,5,2
21038010000996,84,7,5,2
31022010001836,165,5,1,2
12579000000363,381,8,5,0
31022016003394,159,2,2,1


In [11]:
centers = pd.read_csv("../processed_data/center-info.csv", sep=',').set_index('sales_cen_wid')

In [12]:
centers.head()

,region_wid,sales_region_wid
sales_cen_wid,,
66279185761543460,0,32
7889,0,18
623,0,54
233428199161375100,0,45
8269,0,33


In [13]:
items = items.reindex(sales.index.get_level_values(2))
centers = centers.reindex(sales.index.get_level_values(0))

In [15]:
items.head()

,sales_category_wid,productform,energyeffratiing,frequencytype
item_code,,,,
31022010003988,165,5,2,2
31022010003989,165,5,2,2
31022010004009,165,5,2,2
31022010004010,218,5,5,2
31022010005109,218,5,2,2


In [17]:
centers.head()

,region_wid,sales_region_wid
sales_cen_wid,,
-1,0,0
-1,0,0
-1,0,0
-1,0,0
-1,0,0


In [19]:
# 每个产品每天的销量
item_sales = sales.groupby(['item_code'])[sales.columns].sum()

In [20]:
# 每个客户每个产品每天的销量
customer_item_sales = sales.reset_index()
customer_item_sales_index = customer_item_sales[['customer_wid', 'item_code']]
customer_item_sales = sales.groupby(['customer_wid', 'item_code'])[sales.columns].sum()

In [21]:
# 每个销售中心每个产品每天的销量
center_item_sales = sales.reset_index()
center_item_sales_index = center_item_sales[['sales_cen_wid', 'item_code']]
center_item_sales = sales.groupby(['sales_cen_wid', 'item_code'])[sales.columns].sum()

In [22]:
customer_item_sales.head()

creation_date                2018-01-01  2018-01-02  2018-01-03  2018-01-04  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-05  2018-01-06  2018-01-07  2018-01-08  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-09  2018-01-10  2018-01-11  2018-01-12  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-13  2018-01-14  2018-01-15  2018-01-16  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0        15.0   

creation_date                2018-01-17  2018-01-18  2018-01-19  2018-01-20  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-21  2018-01-22  2018-01-23  2018-01-24  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-25  2018-01-26  2018-01-27  2018-01-28  \
customer_wid item_code                                                        
60           12079000000342         0.0         0.0         0.0         0.0   
             12079000000421         0.0         0.0         0.0         0.0   
             12179000000201         0.0         0.0         0.0         0.0   
             12179000000354         0.0         0.0         0.0         0.0   
             12179000001141         0.0         0.0         0.0         0.0   

creation_date                2018-01-29  2018-01-30  2018-01-31  2018-02-01  \
customer_wid item_code                     

In [23]:
center_item_sales.head()

creation_date                 2018-01-01  2018-01-02  2018-01-03  2018-01-04  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-05  2018-01-06  2018-01-07  2018-01-08  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-09  2018-01-10  2018-01-11  2018-01-12  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-13  2018-01-14  2018-01-15  2018-01-16  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-17  2018-01-18  2018-01-19  2018-01-20  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-21  2018-01-22  2018-01-23  2018-01-24  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-25  2018-01-26  2018-01-27  2018-01-28  \
sales_cen_wid item_code                                                        
-1            12179000000201         0.0         0.0         0.0         0.0   
              15111700000430         0.0         0.0         0.0         0.0   
              15111700000431         0.0         0.0         0.0         0.0   
              15111700000452         0.0         0.0         0.0         0.0   
              15111700000453         0.0         0.0         0.0         0.0   

creation_date                 2018-01-29  2018-01-30  2018-01-31  2018-02

# 2 特征工程

In [24]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [ ]:
def prepare_dataset(df, dt, is_train=True, name_prefix=None):
    # 销量统计特征
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, dt, i, i)  # 前i天每个销售中心每个客户每个产品的销量
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的平均一阶差分
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的和
        X['mean_%s' % i] = tmp.mean(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的均值
        X['median_%' % i] = tmp.median(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的中位数
        X['min_%s' % i] = tmp.min(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的最小值
        X['max_%s' % i] = tmp.max(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的最大值
        X['std_%s' % i] = tmp.std(axis=1).values  # 前i天每个销售中心每个客户每个产品的销量的标准差
        
    # 销量统计特征2
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, dt + timedelta(days=-7), i, i)
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的平均一阶差分
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的和
        X['mean_%s_2' % i] = tmp.mean(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的均值
        X['median_%_2' % i] = tmp.median(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的中位数
        X['min_%s_2' % i] = tmp.min(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的最小值
        X['max_%s_2' % i] = tmp.max(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的最大值
        X['std_%s_2' % i] = tmp.std(axis=1).values  # 一周前的前i天每个销售中心每个客户每个产品的销量的标准差
        
    # 有销量的天数特征
    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, dt, i, i)
        X['has_sales_days_in_last_%s' % i] = 